<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Read-SF-Data" data-toc-modified-id="Read-SF-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Read SF Data</a></span></li><li><span><a href="#SF-Data-Analysis" data-toc-modified-id="SF-Data-Analysis-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>SF Data Analysis</a></span><ul class="toc-item"><li><span><a href="#Insight-IDs" data-toc-modified-id="Insight-IDs-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Insight IDs</a></span></li><li><span><a href="#Account-IDs" data-toc-modified-id="Account-IDs-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Account IDs</a></span></li><li><span><a href="#Insight-Type" data-toc-modified-id="Insight-Type-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Insight Type</a></span></li></ul></li><li><span><a href="#UDG-Data" data-toc-modified-id="UDG-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>UDG Data</a></span></li><li><span><a href="#Join-SF-and-UDG-Data" data-toc-modified-id="Join-SF-and-UDG-Data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Join SF and UDG Data</a></span></li><li><span><a href="#Analysis" data-toc-modified-id="Analysis-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Analysis</a></span><ul class="toc-item"><li><span><a href="#Funnel" data-toc-modified-id="Funnel-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Funnel</a></span></li><li><span><a href="#Correlations" data-toc-modified-id="Correlations-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Correlations</a></span></li><li><span><a href="#State" data-toc-modified-id="State-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>State</a></span><ul class="toc-item"><li><span><a href="#State-Distribution:-All-Data-(Opps-and-No-Opps)" data-toc-modified-id="State-Distribution:-All-Data-(Opps-and-No-Opps)-5.3.1"><span class="toc-item-num">5.3.1&nbsp;&nbsp;</span>State Distribution: All Data (Opps and No Opps)</a></span></li><li><span><a href="#State-Distribution:-Opportunities" data-toc-modified-id="State-Distribution:-Opportunities-5.3.2"><span class="toc-item-num">5.3.2&nbsp;&nbsp;</span>State Distribution: Opportunities</a></span></li></ul></li><li><span><a href="#State---Feature-Power" data-toc-modified-id="State---Feature-Power-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>State - Feature Power</a></span><ul class="toc-item"><li><span><a href="#State-Regions" data-toc-modified-id="State-Regions-5.4.1"><span class="toc-item-num">5.4.1&nbsp;&nbsp;</span>State Regions</a></span></li></ul></li><li><span><a href="#Class" data-toc-modified-id="Class-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Class</a></span></li><li><span><a href="#State-and-Class" data-toc-modified-id="State-and-Class-5.6"><span class="toc-item-num">5.6&nbsp;&nbsp;</span>State and Class</a></span></li></ul></li></ul></div>

In [180]:
var_rerun_redshift_query=False
var_rerun_udg_query=False
# dont rerun the salesforce query, instead read the SF data from pre-stored file

In [1]:
%load_ext autoreload
import sys
sys.path.append("/home/ubuntu/Dedupe")

import pandas as pd, re, numpy as np
from itertools import cycle
from collections import defaultdict, Counter
import matplotlib.dates as mdates
import ast
from mypackage import secret
from mypackage.my_functions import *
from mypackage.subcontractor_tag import *
from sqlalchemy import create_engine
from py2neo import Graph,Node,Relationship


from transform import name_transform
name = name_transform.NameTransform()
import pickle as pkl
from tqdm import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
import plotly.offline as pyo
pyo.offline.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import seaborn as sns

In [182]:
engine=create_engine('redshift+psycopg2://'+
                     secret.redshift_secret+'@etl.cscmn6ex0bdf.us-west-2.redshift.amazonaws.com:5439/zlien')
graph = Graph('bolt://read-replica.neo4j.zlien.com', auth=("neo4j", "hiur8OjVubr"))

In [4]:
file_path_sf_query='network_lead_scoring_sf_q_v3.sql'
file_path_sf_data='result_network_lead_scoring_v2.csv'

file_path_input_uuid='input_uuid_for_graph_q.csv'
file_path_udg_data='udg_data.csv'
file_path_taxonomy_state_normalize='../taxonomies/taxonomy_state_normalize.csv'
file_path_taxonomy_state_regions='../taxonomies/taxonomy_state_regions.csv'
file_path_merged_data='merged_data.csv'

In [339]:
df_state_taxonomy=pd.read_csv(file_path_taxonomy_state_normalize)
def def_value(): 
    return "Unknown"
state_dict=defaultdict(def_value) 
for lookup,can_state in df_state_taxonomy[['state_lookup','can_state']].values.tolist():
    state_dict[lookup]=can_state
#state_dict['Texas'.lower()]

def get_can_state(raw_state):
    if raw_state is None or raw_state=='' or str(raw_state)=='nan':
        return 'Missing'
    else:
        return state_dict[re.sub('(?i)[^a-z]','',raw_state.lower())]
def website_status(raw_website):
    if raw_website is None or str(raw_website)=='nan' or str(raw_website)=='0':
        return 'Missing'
    elif re.sub('(?i)[^a-z0-9]','',raw_website)=='':
        return 'Missing'
    else:
        return 'Present'
    
def get_count_percent(df,col_name,drop_na=False):
    if drop_na:
        df_filtered=df[df[col_name].notnull()].copy()
    else:
        df_filtered=df.copy()
        df_filtered[col_name].fillna('Missing',inplace=True)
        df_filtered['insightid'].fillna('Missing',inplace=True)
    #print(df_filtered.shape)
    #print(df_filtered[col_name].value_counts())
    group = df_filtered.groupby(col_name)
    agg = group.aggregate({"insightid": 'count',"accountid": lambda x: x.nunique()}).rename(columns={'insightid': 'Insights', 'accountid': 'Accounts'})
    agg['Insights %']=round(agg['Insights']*100/sum(agg['Insights']),2).astype(str) + '%'
    agg['Accounts %']=round(agg['Accounts']*100/sum(agg['Accounts']),2).astype(str) + '%'
    agg=agg.reset_index()
    col_names=[col_name,'Insights','Insights %','Accounts','Accounts %']
    agg=agg[col_names]
    agg.sort_values(by=col_names[-2], ascending=False,inplace=True)
    return agg

def get_count_percent2(df,col_name,drop_na=False):
    group = df.groupby(col_name)
    agg = group.aggregate({"uuid": 'count',"uuid": lambda x: x.nunique()}).rename(columns={'uuid': 'uuid', 'accountid': 'Accounts'})
    agg['uuid %']=round(agg['uuid']*100/sum(agg['uuid']),2).astype(str) + '%'
    #agg['Accounts %']=round(agg['Accounts']*100/sum(agg['Accounts']),2).astype(str) + '%'
    agg=agg.reset_index()
    col_names=[col_name,'uuid','uuid %']
    agg=agg[col_names]
    agg.sort_values(by=col_names[-2], ascending=False,inplace=True)
    return agg    

def get_count_percent3(df,col_name,drop_na=False):
    group = df.groupby(col_name)
    agg = group.aggregate({"uuid": 'count'})
    agg['uuid %']=round(agg['uuid']*100/sum(agg['uuid']),2).astype(str) + '%'
    #agg['Accounts %']=round(agg['Accounts']*100/sum(agg['Accounts']),2).astype(str) + '%'
    agg=agg.reset_index()
    col_names=[col_name,'uuid','uuid %']
    agg=agg[col_names]
    agg.sort_values(by=col_names[-2], ascending=False,inplace=True)
    return agg    

def collect_states(state):
    list_of_states=list(state)
    list_of_states_unique=[]
    for state in list_of_states:
        if state not in list_of_states_unique:
            list_of_states_unique.append(state)
    if len(list_of_states_unique)>1 and 'Missing' in list_of_states_unique:
        list_of_states_unique.remove('Missing')
    #return list_of_states_unique,len(list_of_states_unique)
    return list_of_states_unique[0]
def collect_websites(website):
    list_of_webs=list(website)
    list_of_webs_unique=[]
    for web in website:
        if website_status(web)=='Present' and web not in list_of_webs_unique:
            list_of_webs_unique.append(web.lower())
    if len(list_of_webs_unique)>0:
        return list_of_webs_unique[0]
    else:
        return ''
def row_style(row):
    if row.state == 'Missing':
        return pd.Series('background-color: red', row.index)
        #else:
        #    return pd.Series('background-color: green', row.index)
    else:
        return pd.Series('', row.index)
def class_remap(class_name,name,wordlist):
    wordlist_str='_'.join(ast.literal_eval(wordlist))
    if class_name=='Electrical Power Generation':
        new_class_name='Electrical'
    elif class_name=='General Requirements':
        new_class_name='no_class'
    elif class_name=='Industry-Specific Manufacturing Equipment' and 'industr' in name.lower():
        new_class_name='no_class'
    elif wordlist_str in ['development','management','management_development','development_management']:
        new_class_name='no_class'
    elif 'mechanical' in wordlist_str :
        new_class_name='no_class'
    else:
        new_class_name=class_name
    return new_class_name

# Read SF Data

In [6]:
if var_rerun_redshift_query:
    fd = open(file_path_sf_query, 'r')
    sqlFile = fd.read()
    fd.close()
    df_input=pd.read_sql(sqlFile, engine)
    print('Rows: %s'%df_input.shape[0])
    df_input.to_csv(file_path_sf_data,encoding='utf_8_sig',index=False)
else:
    df_input=pd.read_csv(file_path_sf_data,parse_dates=['createddate','insight_date','ql_date','opportunity_created_date','customer_date'])
    print('Rows: %s'%df_input.shape[0])
df_input['company_billing_state_norm']=df_input.apply(lambda raw: get_can_state(raw['company_billing_state']),axis=1)
df_input['website_status']=df_input.apply(lambda raw: website_status(raw['website']),axis=1)

Rows: 71916


In [8]:
df_input.to_csv('sf_data_v3.csv',index=False,encoding='utf_8_sig')

# SF Data Analysis

## Insight IDs

In [9]:
#print('-----Insight IDs-----')
print('# Insights: %s'%df_input.shape[0])
print('# Unique Insight Ids: %s'%df_input.insightid.nunique())

# Insights: 71916
# Unique Insight Ids: 35299


## Account IDs

In [10]:
#print('-----Account IDs-----')
print('# Account IDs Fill Rate: %s'%str(round(df_input[df_input['accountid'].notnull()].shape[0]*100/df_input.shape[0],2))+'%')
print('# Identified Company Name Fill Rate: %s'%str(round(df_input[df_input['company_name'].notnull()].shape[0]*100/df_input.shape[0],2))+'%')
print('# Identified UDG IDs Fill Rate: %s'%str(round(df_input[df_input['identified_udg_uuid'].notnull()].shape[0]*100/df_input.shape[0],2))+'%')

print('# Unique Account IDs: %s'%df_input.accountid.nunique())
print('# Unique Company Names: %s'%df_input.company_name.nunique())
print('# Unique Identified UDG IDs: %s'%df_input.identified_udg_uuid.nunique())

# Account IDs Fill Rate: 99.96%
# Identified Company Name Fill Rate: 99.96%
# Identified UDG IDs Fill Rate: 93.84%
# Unique Account IDs: 24308
# Unique Company Names: 24030
# Unique Identified UDG IDs: 20485


## Insight Type

In [11]:
col_name='insight_type'
print_full(get_count_percent(df_input,col_name,drop_na=False))

,insight_type,Insights,Insights %,Accounts,Accounts %
0,Missing,26216,36.45%,19765,75.85%
1,Near Miss,45700,63.55%,6292,24.15%


In [12]:
df_input=df_input.sort_values(by=['ql_date'],ascending=[False]).reset_index(drop=True)
df_input_grouped=df_input.groupby('identified_udg_uuid').\
aggregate({\
           #'company_name': [lambda x: sorted(list(set(x)),key=len,reverse=True),lambda x: len(list(set(x)))],
           'company_name': lambda x: sorted(list(set(x)),key=len,reverse=True)[0],
           #'company_billing_state_norm': [lambda x: collect_states(x)[0],lambda x: collect_states(x)[1]],
           'company_billing_state_norm': lambda x: collect_states(x),
           'website': lambda x: collect_websites(x),
           'insightid': ['count', 'nunique'],
           'is_customer':lambda x: 1 if 'Yes' in set(x) else 0,
           'is_opportunity':lambda x: 1 if 'Yes' in set(x) else 0,
           'is_sales_accepted':lambda x: 1 if 'Yes' in set(x) else 0,
           'sal_date':[lambda x: min([i for i in x if str(i)!='nan']) 
                       if len([i for i in x if str(i)!='nan'])>0 else '',
                       lambda x: max([i for i in x if str(i)!='nan']) 
                       if len([i for i in x if str(i)!='nan'])>0 else ''],
           'ql_date':['min','max'],
           'contact_stage': lambda x: dict(Counter(x)),
           'sales_rejected_reason': lambda x: dict(Counter(x)),
           'accountid':lambda x: list(set(x))
          }).reset_index()
#df_input_grouped.columns=['uuid','company_name','company_name_count','insights','insights_unique','is_customer','is_opportunity','ql_date_earliest','ql_date_latest']
#df_input_grouped.columns=['uuid','company_name','states','states_count','insights','insights_unique','is_customer','is_opportunity','ql_date_earliest','ql_date_latest']
df_input_grouped.columns=['uuid','company_name','state','website',
                          'insights','insights_unique','is_customer',
                          'is_opportunity','is_sales_accepted','sa_date_earliest','sa_date_latest','ql_date_earliest','ql_date_latest',
                          'contact_stage','sales_rejected_reason','account_ids']
df_state_regions=pd.read_csv(file_path_taxonomy_state_regions,encoding='utf_8-sig')
df_input_grouped=df_input_grouped.merge(df_state_regions[['state_code','region']],how='left',left_on='state',right_on='state_code').drop(columns=['state_code']).rename(columns={'region': 'state_region'})
df_input_grouped['state_region'].fillna('Missing',inplace=True)
df_input_grouped=df_input_grouped[['uuid','company_name','state_region','state','website',
                          'insights','insights_unique','is_customer',
                          'is_opportunity','is_sales_accepted','sa_date_earliest','sa_date_latest',
                           'ql_date_earliest','ql_date_latest',
                          'contact_stage','sales_rejected_reason','account_ids']]

# UDG Data

In [198]:
q="""UNWIND $ids as row
    MATCH (company:Company)
    WHERE company.uuid=row
    WITH company
    //find projects
    //count projects
    optional match (company)-[:WORKED_ON]-(proj:Project)
    where proj.creation_date>='2019-10-01'
    with company, 
        count(distinct proj) as project_count
    //get project breakdown by state
    optional match (company)-[:WORKED_ON]-(proj:Project)-[:LOCATED_IN]-(proj_address:State)
    where proj.creation_date>='2019-10-01'
    with company, 
        project_count, 
        proj_address.name as state_name, 
        size(collect(proj_address.name)) as state_count order by state_name
    with collect([state_name,state_count]) as project_states, 
        company, 
        project_count
    //count any documents sent
    optional match (company)-[:WORKED_ON]-(proj:Project)-[:SENT_ON]-(document:Document)
    where proj.creation_date>='2019-10-01'
    and document.type in ["Lien","Notice of Intent"]
    with company, 
        project_count, 
        count(document) as doc_count, 
        project_states
    optional match (company)-[:WORKED_ON]-(proj:Project)-[:SENT_ON]-(document:Document)-[:SENT_BY]-(sender:Company)
    where proj.creation_date>='2019-10-01'
    and document.type in ["Lien","Notice of Intent"]
    with company, 
        project_count, 
        doc_count, 
        count(distinct sender) as sender_count, 
        project_states
    match (company)-[:LOCATED_IN]-(companylocation:CompanyLocation)-[:LOCATED_IN]-(locationaddress:Address)
    optional match (locationaddress)-[:LOCATED_IN]-(zip:Zip)
        with company,
        project_count,
        doc_count,
        sender_count,
        project_states,
        count(distinct companylocation) as location_count,
        collect(DISTINCT [zip.city,zip.name,zip.state]) as company_locations
    return distinct company.uuid AS uuid, 
        company.name AS company_name, 
        project_count, 
        doc_count, 
        sender_count,
        location_count,
        company_locations,
        project_states
    """
df_result=graph.run(q,ids=list(df_input_grouped['uuid'])).to_data_frame()
df_result.shape

- Unique UUIDs to Query:           20485
- UUIDs Found in the Graph:        18126
- Unique UUIDs Found in the Graph: 18126

# Join SF and UDG Data

-  Rows in SF-UDG Joined Data: 18126

# Analysis

In [211]:
df_input_grouped1=pd.read_csv(file_path_merged_data,encoding='utf_8_sig')

## Funnel

In [212]:
print('Unique Accounts: %s'%df_input.accountid.nunique())
print('Unique Accounts with UDG IDs: %s'%df_input_grouped.uuid.nunique())
print('Unique Accounts with UDG Data: %s'%df_udg_input.uuid.nunique())
print('Unique Accounts with Overlapping UDG Data: %s'%df_input_grouped1.uuid.nunique())

Unique Accounts: 24308
Unique Accounts with UDG IDs: 20485
Unique Accounts with UDG Data: 18126
Unique Accounts with Overlapping UDG Data: 18126


In [213]:
count_quailified=df_input_grouped1.shape[0]
count_sales_accepted=df_input_grouped1[df_input_grouped1['is_sales_accepted']==1].shape[0]
count_opportunities=df_input_grouped1[df_input_grouped1['is_opportunity']==1].shape[0]
count_customer=df_input_grouped1[df_input_grouped1['is_customer']==1].shape[0]
print('Qualified (UUIDs): %s'%count_quailified)
print('Sales Accepted:    %s'%count_sales_accepted)
print('Opportunities:     %s'%count_opportunities)
print('Customers:         %s'%count_customer)


Qualified (UUIDs): 18126
Sales Accepted:    13985
Opportunities:     1208
Customers:         130


In [214]:
print('Sales Not Accepted to Opportunities')
print_full(get_count_percent3(df_input_grouped1[df_input_grouped1['is_sales_accepted']==0],'is_opportunity',drop_na=False))


Sales Not Accepted to Opportunities


,is_opportunity,uuid,uuid %
0,0,4036,97.46%
1,1,105,2.54%


<img src="../../image_art/idea_insight.png" alt="Insight" style="float: left;width: 80px;"/>

- 105 Opportunities are marked as `Sales Not Accepted`
- Marking this 105 Opportunities as `Sales Accepted`

In [215]:
df_input_grouped1['is_sales_accepted']=df_input_grouped1.apply(lambda x:x['is_opportunity'] if x['is_opportunity']==1 else x['is_sales_accepted'],axis=1)
print('Revised Numbers')
count_quailified=df_input_grouped1.shape[0]
count_sales_accepted=df_input_grouped1[df_input_grouped1['is_sales_accepted']==1].shape[0]
count_opportunities=df_input_grouped1[df_input_grouped1['is_opportunity']==1].shape[0]
count_customer=df_input_grouped1[df_input_grouped1['is_customer']==1].shape[0]
print('Qualified (UUIDs): %s'%count_quailified)
print('Sales Accepted:    %s'%count_sales_accepted)
print('Opportunities:     %s'%count_opportunities)
print('Customers:         %s'%count_customer)


Revised Numbers
Qualified (UUIDs): 18126
Sales Accepted:    14090
Opportunities:     1208
Customers:         130


In [216]:
print('Qualified to Sales Accepted')
print_full(get_count_percent3(df_input_grouped1,'is_sales_accepted',drop_na=False))
print('--------------------------------')
print('')
print('Qualified to Opportunities')
print_full(get_count_percent3(df_input_grouped1,'is_opportunity',drop_na=False))
print('--------------------------------')
print('')
print('Sales Accepted to Opportunities')
print_full(get_count_percent3(df_input_grouped1[df_input_grouped1['is_sales_accepted']==1],'is_opportunity',drop_na=False))
print('--------------------------------')
print('')


print('Qualified to Customer')
print_full(get_count_percent3(df_input_grouped1,'is_customer',drop_na=False))
print('--------------------------------')
print('')
print('Sales Accepted to Customer')
print_full(get_count_percent3(df_input_grouped1[df_input_grouped1['is_sales_accepted']==1],'is_customer',drop_na=False))
print('--------------------------------')
print('')
print('Opportunity to Customer')
print_full(get_count_percent3(df_input_grouped1[df_input_grouped1['is_opportunity']==1],'is_customer',drop_na=False))
#print('--------------------------------')
#print('')

fig = go.Figure()
fig = go.Figure(go.Funnel(
    y = ["Qualified Accounts","Sales Accepted","Opportunities", "Customers"],
    x = [count_quailified,count_sales_accepted,count_opportunities,count_customer],
    textposition = "inside",
    textinfo = "value+percent initial",
    opacity = 0.65, marker = {"color": ["blue","deepskyblue", "lightsalmon","teal"],
    "line": {"width": [2, 1, 1], "color": ["wheat","wheat", "wheat", "wheat"]}},
    connector = {"line": {"color": "royalblue", "dash": "dot", "width": 3}})
    )
fig.show()

Qualified to Sales Accepted


,is_sales_accepted,uuid,uuid %
1,1,14090,77.73%
0,0,4036,22.27%


--------------------------------

Qualified to Opportunities


,is_opportunity,uuid,uuid %
0,0,16918,93.34%
1,1,1208,6.66%


--------------------------------

Sales Accepted to Opportunities


...

,is_customer,uuid,uuid %
0,0,17996,99.28%
1,1,130,0.72%


--------------------------------

Sales Accepted to Customer


,is_customer,uuid,uuid %
0,0,13960,99.08%
1,1,130,0.92%


--------------------------------

Opportunity to Customer


,is_customer,uuid,uuid %
0,0,1078,89.24%
1,1,130,10.76%


In [217]:
df_input_grouped1=df_input_grouped1[df_input_grouped1['is_sales_accepted']==1]
print('Sales Accepted Leads: %s'%df_input_grouped1.shape[0])

Sales Accepted Leads: 14090


## Correlations

In [218]:
numerical_variables=['insights','project_count','doc_count',
       'sender_count', 'location_count','subscriber_companies', 'retail_companies', 'non_users']

corr_list={}
for col in numerical_variables:
    corr_list[col]=round(df_input_grouped1[col].corr(df_input_grouped1['is_opportunity']),4)
df_corr=pd.DataFrame.from_dict(corr_list,orient ='index').reset_index()
df_corr.columns=['variable','correlation']
df_corr=df_corr.sort_values(by=['correlation'],ascending=[False])
df_corr

,variable,correlation
3,sender_count,0.0311
0,insights,0.0297
6,retail_companies,0.0241
2,doc_count,0.0200
4,location_count,0.0147
7,non_users,0.0064
1,project_count,0.0042
5,subscriber_companies,NaN


<img src="../../image_art/idea_insight.png" alt="Insight" style="float: left;width: 100px;"/>

- Very weak correlation between opportunities and the variables
- Highest correlation is for __sender count (0.0310)__ followed by __insights count (0.0297)__

## State

### State Distribution: All Data (Opps and No Opps)

In [223]:
#print('-----State-----')
print('# Company Billing State Fill Rate: %s'%str(round(df_input_grouped1[~df_input_grouped1['state_code'].isin(['Missing','Unknown'])].shape[0]*100/df_input_grouped1.shape[0],2))+'%')
print('# Company Billing State Missing: %s'%str(round(df_input_grouped1[df_input_grouped1['state_code'].isin(['Missing'])].shape[0]*100/df_input_grouped1.shape[0],2))+'%')
print('# Company Billing State Invalid: %s'%str(round(df_input_grouped1[df_input_grouped1['state_code'].isin(['Unknown'])].shape[0]*100/df_input_grouped1.shape[0],2))+'%')


pivot_norm_state=df_input_grouped1['state'].value_counts(dropna='False').to_frame().reset_index()
pivot_norm_state.columns=['state','count']
pivot_norm_state=pivot_norm_state.merge(df_state_regions[['state_code','state_name']],how='left',left_on='state',right_on='state_name').drop(['state_name'],axis=1)
pivot_norm_state.state_code.fillna('Missing',inplace=True)
pivot_norm_state=pivot_norm_state[['state','state_code','count']]
pivot_norm_state['%']=round(pivot_norm_state['count']*100/sum(pivot_norm_state['count']),2)
pivot_norm_state=pivot_norm_state.sort_values(by=['%'],ascending=[False])
pivot_norm_state['%']=pivot_norm_state['%'].astype(str)+'%'

pivot_norm_state_opp=df_input_grouped1[df_input_grouped1['is_opportunity']==1]['state'].value_counts(dropna='False').to_frame().reset_index()
pivot_norm_state_opp.columns=['state','count']
pivot_norm_state_opp=pivot_norm_state_opp.merge(df_state_regions[['state_code','state_name']],how='left',left_on='state',right_on='state_name').drop(['state_name'],axis=1)
pivot_norm_state_opp.state_code.fillna('Missing',inplace=True)
pivot_norm_state_opp=pivot_norm_state_opp[['state','state_code','count']]
pivot_norm_state_opp['%']=round(pivot_norm_state_opp['count']*100/sum(pivot_norm_state_opp['count']),2)
pivot_norm_state_opp=pivot_norm_state_opp.sort_values(by=['%'],ascending=[False])
pivot_norm_state_opp['%']=pivot_norm_state_opp['%'].astype(str)+'%'


print('# Unique States: %s'%pivot_norm_state.shape[0])
#print('Top 10 Raw State')
#col_name='company_billing_state'
#print_full(get_count_percent1(col_name).head(10))

print('Top 10 States')
pivot_norm_state.head(10).style.apply(row_style, axis=1).set_table_attributes("style='display:inline'")
pivot_norm_state_opp.head(10).style.apply(row_style, axis=1).set_table_attributes("style='display:inline'")


df1_styler = pivot_norm_state.head(10).style.apply(row_style, axis=1).set_table_attributes("style='display:inline'").set_caption('All Leads')
df2_styler = pivot_norm_state_opp.head(10).style.apply(row_style, axis=1).set_table_attributes("style='display:inline'").set_caption('Oppertunities')
from IPython.display import display_html 
display_html(df1_styler._repr_html_()+'<<<<>>>>>'+df2_styler._repr_html_(), raw=True)

# Company Billing State Fill Rate: 85.14%
# Company Billing State Missing: 14.74%
# Company Billing State Invalid: 0.12%
# Unique States: 51
Top 10 States


,state,state_code,count,%
0,Texas,TX,4706,33.4%
1,California,CA,2613,18.55%
2,Missing,Missing,2104,14.93%
3,Florida,FL,886,6.29%
4,Arizona,AZ,388,2.75%
5,Georgia,GA,334,2.37%
6,Nevada,NV,317,2.25%
7,Colorado,CO,295,2.09%
8,New York,NY,233,1.65%
9,Washington,WA,225,1.6%


In [230]:
fig = go.Figure(go.Choropleth(
    locations=pivot_norm_state['state_code'],
    z=pivot_norm_state['count'],
    locationmode='USA-states',
    #dragmode= False,
    text=round(pivot_norm_state['count']*100/sum(pivot_norm_state['count']),2),
                          ))

fig.update_layout(
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)  

### State Distribution: Opportunities

In [231]:
fig = go.Figure(go.Choropleth(
    locations=pivot_norm_state_opp['state_code'],
    z=pivot_norm_state_opp['count'],
    locationmode='USA-states',
    #dragmode= False,
    text=round(pivot_norm_state_opp['count']*100/sum(pivot_norm_state_opp['count']),2),
                          ))

fig.update_layout(
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)  

## State - Feature Power

In [345]:
df_pivot_states=pd.crosstab(index=df_input_grouped1['state'], columns=df_input_grouped1['is_opportunity'], values=df_input_grouped1['uuid'], aggfunc=pd.Series.nunique).reset_index()
df_pivot_states.fillna(0,inplace=True)

df_pivot_states.columns=['state','opportunity_no','opportunity_yes']
df_pivot_states=df_pivot_states.merge(df_state_regions[['state_code','state_name']],how='left',left_on='state',right_on='state_name').drop(['state_name'],axis=1)
df_pivot_states.state_code.fillna('Missing',inplace=True)
df_pivot_states['total']=df_pivot_states['opportunity_no']+df_pivot_states['opportunity_yes']

base_rate=round(sum(df_pivot_states.opportunity_yes)/sum(df_pivot_states.total)*100,2)
df_pivot_states['opp_rate']=round(df_pivot_states['opportunity_yes']*100/df_pivot_states['total'],4)
df_pivot_states['feature_power']=df_pivot_states['opp_rate']/base_rate
df_pivot_states.sort_values(by=['feature_power','total'],ascending=[False,False],inplace=True)
df_pivot_states=df_pivot_states[['state','state_code','opportunity_no','opportunity_yes','total','opp_rate','feature_power']]


df_pivot_states['opportunity_no']=df_pivot_states['opportunity_no'].astype(int)
df_pivot_states['opportunity_yes']=df_pivot_states['opportunity_yes'].astype(int)
df_pivot_states['total']=df_pivot_states['total'].astype(int)



print('States with Positive Feature Power - Top 10')
sel_cols=['state','opportunity_no','opportunity_yes','total','opp_rate','feature_power']
print_full(df_pivot_states[df_pivot_states['feature_power']>1][sel_cols].head(10))

print('States with Negative Feature Power - Bottom 10')
print_full(df_pivot_states[df_pivot_states['feature_power']<1][sel_cols].tail(10))


fig = go.Figure(go.Choropleth(
    locations=df_pivot_states['state_code'],
    z=df_pivot_states['feature_power'],
    locationmode='USA-states',
    colorscale=["red","red","yellow","orange", "teal","teal","teal"],
    #dragmode= False,
    #textinfo = "value",
    text = df_pivot_states['state_code'],
    #text=round(pivot_norm_state['Count']*100/sum(pivot_norm_state['Count']),2),
                          ))

fig.update_layout(
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        #showlakes=False, # lakes
        #lakecolor='rgb(255, 255, 255)'
    ),
)  

States with Positive Feature Power - Top 10


,state,opportunity_no,opportunity_yes,total,opp_rate,feature_power
39,Rhode Island,5,2,7,28.57,3.33
26,Montana,4,1,5,20.00,2.33
16,Kentucky,15,3,18,16.67,1.94
34,North Dakota,8,1,9,11.11,1.30
49,Wisconsin,50,6,56,10.71,1.25
4,California,2342,271,2613,10.37,1.21
9,Georgia,301,33,334,9.88,1.15
33,North Carolina,192,21,213,9.86,1.15
5,Colorado,267,28,295,9.49,1.11
23,Missing,1905,199,2104,9.46,1.10


States with Negative Feature Power - Bottom 10


,state,opportunity_no,opportunity_yes,total,opp_rate,feature_power
11,Idaho,19,0,19,0.00,0.00
29,New Hampshire,9,0,9,0.00,0.00
7,Delaware,8,0,8,0.00,0.00
1,Alaska,5,0,5,0.00,0.00
18,Maine,4,0,4,0.00,0.00
41,South Dakota,4,0,4,0.00,0.00
10,Hawaii,2,0,2,0.00,0.00
48,West Virginia,2,0,2,0.00,0.00
45,Vermont,1,0,1,0.00,0.00
50,Wyoming,1,0,1,0.00,0.00


### State Regions

<img src="../../image_art/us_regions.png?1" alt="Insight" style="float: left;width: 900px;"/>

In [233]:
df_pivot_state_region_all=pd.crosstab(index=df_input_grouped1['state_region'], columns=df_input_grouped1['is_opportunity'], values=df_input_grouped1['uuid'], aggfunc=pd.Series.nunique).reset_index()
df_pivot_state_region_all.columns=['state_region','opportunity_no','opportunity_yes']
df_pivot_state_region_all['total']=df_pivot_state_region_all['opportunity_no']+df_pivot_state_region_all['opportunity_yes']

base_rate=round(sum(df_pivot_state_region_all.opportunity_yes)/sum(df_pivot_state_region_all.total)*100,2)
df_pivot_state_region_all['opp_rate']=round(df_pivot_state_region_all['opportunity_yes']*100/df_pivot_state_region_all['total'],4)
df_pivot_state_region_all['feature_power']=df_pivot_state_region_all['opp_rate']/base_rate
df_pivot_state_region_all.sort_values(by=['feature_power','total'],ascending=[False,False],inplace=True)
df_pivot_states['opportunity_no']=df_pivot_states['opportunity_no'].astype(int)
df_pivot_states['opportunity_yes']=df_pivot_states['opportunity_yes'].astype(int)
df_pivot_states['total']=df_pivot_states['total'].astype(int)
print_full(df_pivot_state_region_all)



,state_region,opportunity_no,opportunity_yes,total,opp_rate,feature_power
5,West,3275,348,3623,9.61,1.12
1,Missing,1905,199,2104,9.46,1.10
4,Southwest,4775,441,5216,8.45,0.99
3,Southeast,1768,148,1916,7.72,0.90
0,Midwest,631,45,676,6.66,0.78
2,Northeast,528,27,555,4.86,0.57


## Class

In [347]:
df_input_grouped1['l1_csi_class_name']=df_input_grouped1.apply(lambda x:class_remap(x['level_1_csi_class_name'],x['company_name'],x['word_list']),axis=1)
df_pivot_class=pd.crosstab(index=df_input_grouped1['l1_csi_class_name'], columns=df_input_grouped1['is_opportunity'], values=df_input_grouped1['uuid'], aggfunc=pd.Series.nunique).reset_index()
df_pivot_class.fillna(0,inplace=True)
df_pivot_class.columns=['class','opportunity_no','opportunity_yes']
df_pivot_class['total']=df_pivot_class['opportunity_no']+df_pivot_class['opportunity_yes']
base_rate=round(sum(df_pivot_class.opportunity_yes)/sum(df_pivot_class.total)*100,2)
df_pivot_class['opp_rate']=round(df_pivot_class['opportunity_yes']*100/df_pivot_class['total'],4)
df_pivot_class['feature_power']=df_pivot_class['opp_rate']/base_rate
df_pivot_class.sort_values(by=['feature_power','total'],ascending=[False,False],inplace=True)
df_pivot_class=df_pivot_class.reset_index(drop=True)
df_pivot_class['opportunity_no']=df_pivot_class['opportunity_no'].astype(int)
df_pivot_class['opportunity_yes']=df_pivot_class['opportunity_yes'].astype(int)
df_pivot_class['total']=df_pivot_class['total'].astype(int)


print('Classes with Positive Feature Power - Top 10')
sel_cols=['class','opportunity_no','opportunity_yes','total','opp_rate','feature_power']
print_full(df_pivot_class[df_pivot_class['feature_power']>1][sel_cols].head(10))

print('Classes with Negative Feature Power - Bottom 10')
print_full(df_pivot_class[df_pivot_class['feature_power']<1].sort_values(by=['feature_power','total'])[sel_cols].head(10))



Classes with Positive Feature Power - Top 10


,class,opportunity_no,opportunity_yes,total,opp_rate,feature_power
0,"Process Gas and Liquid Handling, Purification, and Storage Equipment",20,4,24,16.67,1.94
1,"Process Heating, Cooling, and Drying Equipment",37,6,43,13.95,1.63
2,Masonry,179,28,207,13.53,1.58
3,Utilities,133,20,153,13.07,1.53
4,Thermal and Moisture Protection,358,53,411,12.90,1.50
5,Earthwork,123,18,141,12.77,1.49
6,Waterway and Marine Construction,49,7,56,12.50,1.46
7,Specialties,121,17,138,12.32,1.44
8,Fire Suppression,226,30,256,11.72,1.37
9,Plumbing,468,58,526,11.03,1.29


Classes with Negative Feature Power - Bottom 10


,class,opportunity_no,opportunity_yes,total,opp_rate,feature_power
33,Water and Wastewater Equipment,20,0,20,0.00,0.00
32,Industry-Specific Manufacturing Equipment,50,0,50,0.00,0.00
31,Process Integration,30,1,31,3.23,0.38
30,Transportation,28,1,29,3.45,0.40
29,Pollution and Waste Control Equipment,24,1,25,4.00,0.47
28,Electronic Safety and Security,47,2,49,4.08,0.48
27,Communications,74,4,78,5.13,0.60
26,Furnishings,239,15,254,5.91,0.69
25,Procurement and Contracting Requirements,72,5,77,6.49,0.76
24,Integrated Automation,53,4,57,7.02,0.82


## State and Class

In [349]:
df_pivot_state_class=pd.crosstab(index=[df_input_grouped1['l1_csi_class_name'],df_input_grouped1['state_region']], columns=df_input_grouped1['is_opportunity'], values=df_input_grouped1['uuid'], aggfunc=pd.Series.nunique).reset_index()
df_pivot_state_class.columns=['class','state_region','opportunity_no','opportunity_yes']
df_pivot_state_class.fillna(0,inplace=True)

df_pivot_state_class['total']=df_pivot_state_class['opportunity_no']+df_pivot_state_class['opportunity_yes']

base_rate=round(sum(df_pivot_state_class.opportunity_yes)/sum(df_pivot_state_class.total)*100,2)
df_pivot_state_class['opp_rate']=round(df_pivot_state_class['opportunity_yes']*100/df_pivot_state_class['total'],4)
df_pivot_state_class['feature_power']=df_pivot_state_class['opp_rate']/base_rate
df_pivot_state_class.sort_values(by=['feature_power','total'],ascending=[False,False],inplace=True)


print('Class-States with Positive Feature Power - Top 10')
sel_cols=['class','opportunity_no','opportunity_yes','total','opp_rate','feature_power']
print_full(df_pivot_state_class[df_pivot_state_class['feature_power']>1][sel_cols].head(10))

print('Class-States with Negative Feature Power - Bottom 10')
print_full(df_pivot_state_class[df_pivot_state_class['feature_power']<1].sort_values(by=['feature_power','total'])[sel_cols].head(10))





Class-States with Positive Feature Power - Top 10


,class,opportunity_no,opportunity_yes,total,opp_rate,feature_power
134,Process Integration,0.00,1.00,1.00,100.00,11.67
187,"Wood, Plastics, and Composites",3.00,2.00,5.00,40.00,4.67
182,Waterway and Marine Construction,6.00,3.00,9.00,33.33,3.89
151,Specialties,4.00,2.00,6.00,33.33,3.89
2,Communications,2.00,1.00,3.00,33.33,3.89
169,Utilities,2.00,1.00,3.00,33.33,3.89
123,"Process Gas and Liquid Handling, Purification, and Storage Equipment",5.00,2.00,7.00,28.57,3.33
36,Equipment,6.00,2.00,8.00,25.00,2.92
84,Integrated Automation,6.00,2.00,8.00,25.00,2.92
91,Masonry,3.00,1.00,4.00,25.00,2.92


Class-States with Negative Feature Power - Bottom 10


,class,opportunity_no,opportunity_yes,total,opp_rate,feature_power
85,Integrated Automation,1.00,0.00,1.00,0.00,0.00
95,Material Processing and Handling Equipment,1.00,0.00,1.00,0.00,0.00
135,Process Integration,1.00,0.00,1.00,0.00,0.00
163,Transportation,1.00,0.00,1.00,0.00,0.00
165,Transportation,1.00,0.00,1.00,0.00,0.00
171,Utilities,1.00,0.00,1.00,0.00,0.00
175,Water and Wastewater Equipment,1.00,0.00,1.00,0.00,0.00
176,Water and Wastewater Equipment,1.00,0.00,1.00,0.00,0.00
181,Waterway and Marine Construction,1.00,0.00,1.00,0.00,0.00
30,Electronic Safety and Security,2.00,0.00,2.00,0.00,0.00


In [356]:
print('Class-States with Negative Feature Power - Top 10 Significant')
print_full(df_pivot_state_class[(df_pivot_state_class['total']>30) & (df_pivot_state_class['feature_power']>1)].head(10))

print('Class-States with Negative df_pivot_state_class Power - Bottom 10 Significant')
print_full(df_pivot_state_class[(df_pivot_state_class['total']>30) & (df_pivot_state_class['feature_power']<1)].sort_values(by=['feature_power','total']).head(10))


Class-States with Negative Feature Power - Top 10 Significant


,class,state_region,opportunity_no,opportunity_yes,total,opp_rate,feature_power
94,Masonry,West,38.00,10.00,48.00,20.83,2.43
64,Fire Suppression,Southwest,73.00,16.00,89.00,17.98,2.10
23,Earthwork,West,28.00,6.00,34.00,17.65,2.06
160,Thermal and Moisture Protection,Southeast,38.00,8.00,46.00,17.39,2.03
53,Exterior Improvements,West,39.00,8.00,47.00,17.02,1.99
162,Thermal and Moisture Protection,West,90.00,18.00,108.00,16.67,1.94
41,Equipment,West,26.00,5.00,31.00,16.13,1.88
90,Masonry,Missing,32.00,6.00,38.00,15.79,1.84
174,Utilities,West,32.00,6.00,38.00,15.79,1.84
102,Metals,Missing,28.00,5.00,33.00,15.15,1.77


Class-States with Negative df_pivot_state_class Power - Bottom 10 Significant


,class,state_region,opportunity_no,opportunity_yes,total,opp_rate,feature_power
54,Finishes,Midwest,35.00,0.00,35.00,0.00,0.00
16,Conveying Equipment,Southwest,32.00,1.00,33.00,3.03,0.35
49,Exterior Improvements,Missing,30.00,1.00,31.00,3.23,0.38
193,no_class,Northeast,292.00,11.00,303.00,3.63,0.42
57,Finishes,Southeast,82.00,4.00,86.00,4.65,0.54
73,"Heating, Ventilating, and Air Conditioning (HVAC)",Missing,40.00,2.00,42.00,4.76,0.56
71,Furnishings,West,57.00,3.00,60.00,5.00,0.58
150,Special Construction,West,33.00,2.00,35.00,5.71,0.67
65,Fire Suppression,West,76.00,5.00,81.00,6.17,0.72
194,no_class,Southeast,929.00,65.00,994.00,6.54,0.76
